In [131]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.utils import getDataFilesSorted

import config

In [132]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 3
TOTAL_BUDGET = 1000

In [133]:
current_team_names = {"\u0110or\u0111e Petrovi\u0107",
                      "Alisson Becker",
                      "Gabriel dos Santos Magalhães",
                      "Matty Cash",
                      "Marcos Senesi Bar\u00f3n",
                      "Adrien Truffert",
                      "Micky van de Ven",
                      "Yankuba Minteh",
                      "Jaidon Anthony",
                      "Idrissa Gana Gueye",
                      "Alex Iwobi",
                      "Bruno Borges Fernandes",
                      "Erling Haaland",
                      "Jarrod Bowen",
                      "Jean-Philippe Mateta",
                      }

In [134]:
# TODO: Change how files are loaded

with open(teams_filename, "r") as f:
    tempJson: list[dict] = json.load(f)

actualJson = tempJson[SELECTED_MODEL_INDEX]

In [135]:
all_player_data = pd.DataFrame.from_records(actualJson["players"])

In [136]:
print(all_player_data)

     clean_sheets  cost  expected_goals  form  gameweek  ict_index   id  \
0             7.0    58            0.00   6.0        11       16.8    1   
1             0.0    42            0.00   0.0        11        0.0    2   
2             0.0    40            0.00   0.0        11        0.0    3   
3             0.0    40            0.00   0.0        11        0.0    4   
4             7.0    66            0.82  11.0        11       42.8    5   
..            ...   ...             ...   ...       ...        ...  ...   
743           0.0    45            0.00   0.3        11        0.0  744   
744           0.0    40            0.00   0.0        11        0.0  745   
745           0.0    45            0.00   0.0        11        0.0  746   
746           0.0    40            0.00   0.0        11        0.0  747   
747           0.0    50            0.00   0.0        11        0.0  748   

                             name opposing_team  play_percent  \
0               David Raya Martín 

In [137]:
currentTeamPlayers = all_player_data.loc[all_player_data["name"].isin(current_team_names)]


for player in current_team_names:
    if(player not in currentTeamPlayers["name"].values):
        raise ValueError(f"player '{player}' not found")

In [138]:
current_team = Team.fromDataFrame(currentTeamPlayers)

In [139]:
current_team_cost = current_team.getTotalCost()
current_team_cost

983

In [140]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)

all_data = all_data[SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame.from_records(all_data["team"])

In [141]:
selected_team = Team.fromDataFrame(selected_team_df)

In [142]:
display(selected_team)

In [143]:
new_players = selected_team - current_team
print(new_players)


Total Score: 85.071

Goalkeepers:
- David Raya Martín	Score: 5.58	Cost: 58	Fixture Difficulty: 0.0
Defenders:
- Marc Cucurella Saseta	Score: 7.6	Cost: 61	Fixture Difficulty: 0.0
- Reece James (Captain) 	Score: 11.52	Cost: 55	Fixture Difficulty: 0.0
- Quilindschy Hartman	Score: 6.55	Cost: 40	Fixture Difficulty: 0.0
Attackers:
- Danny Welbeck	Score: 7.64	Cost: 65	Fixture Difficulty: 0.0
- Eli Junior Kroupi	Score: 5.38	Cost: 46	Fixture Difficulty: 0.0
Midfielders:
- Bryan Mbeumo	Score: 7.29	Cost: 84	Fixture Difficulty: 0.0
- Mohamed Salah	Score: 8.44	Cost: 142	Fixture Difficulty: 0.0
- Ryan Sessegnon	Score: 8.76	Cost: 54	Fixture Difficulty: 0.0
- Carlos Henrique Casimiro (Vice Captain) 	Score: 9.18	Cost: 55	Fixture Difficulty: 0.0
- Pedro Lomba Neto	Score: 7.14	Cost: 71	Fixture Difficulty: 0.0


In [144]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -14


In [145]:
#HEURISTIC = "combined"
#MODE = SolverMode.CHEAPEST_FIRST

In [146]:
# all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(1.9096956524813034, 1.9096956524813034)

In [147]:
matrix = FixtureDifficultyMatrix()
matrix.precomputeFixtureDifficulty(0, config.CURRENT_GAMEWEEK+1, 3, config.CURRENT_SEASON, 1.0)

new_players.recalculateFixtureDifficulty(matrix)
current_team.recalculateFixtureDifficulty(matrix)
# new_players.calculateScore(HEURISTIC)

# selected_team.recalculateFixtureDifficulty(matrix)
# selected_team.calculateScore(HEURISTIC)

# current_team.recalculateFixtureDifficulty(matrix)
# current_team.calculateScore(HEURISTIC)

In [148]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [149]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 39.780375


In [150]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    # TODO: Store Team players in sorted tree instead of list
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [151]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [152]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [153]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [154]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team, pMatrix: FixtureDifficultyMatrix):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    transferData.updateFixtureDifficulties(pMatrix)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    

    newTeam.recalculateFixtureDifficulty(pMatrix)

    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [155]:
def checkBenchBoost(pTeam: BenchTeam):
    halfNumGameweeks: int = config.MAX_GAMEWEEKS // 2
    remainingWeeks = halfNumGameweeks - config.CURRENT_GAMEWEEK
    percentLeft: float = remainingWeeks / halfNumGameweeks
    totalFixtureDifficulty: float = 0.0
    for player in pTeam.getBenchPlayerList():
        totalFixtureDifficulty += player.getCurrentDifficulty()
    avgDifficulty: float = totalFixtureDifficulty / len(pTeam.getBenchPlayerList())
    DEFAULT_THRESHOLD = 0.35
    actualThreshold = ((1.0 - percentLeft) * config.MAX_CHIP_DECAY) + DEFAULT_THRESHOLD
    if (avgDifficulty <= actualThreshold):
        toPrint = "Bench boost is suggested. Threshold: {:.3f} Average Fixture Difficulty: {:.3f}".format(actualThreshold, avgDifficulty)
        display(toPrint)
    else:
        toPrint = "Bench boost not needed. Threshold: {:.3f} Average Fixture Difficulty: {:.3f}".format(actualThreshold, avgDifficulty)
        display(toPrint)

In [156]:
newTeam = getNewTeam(current_team, selected_team, new_players, matrix)
newTeam = newTeam.toBenchTeam()
display(newTeam)
display(f"New total score: {newTeam.getTotalScore()}")
display(f"New total cost: {newTeam.getTotalCost()}")
checkBenchBoost(newTeam)

Best transfer:
Transfer from Adrien Truffert -> Reece James:
Old player: Adrien Truffert	Score: 1.93	Cost: 45	Fixture Difficulty: 0.375
New player: Reece James (Captain) 	Score: 8.4	Cost: 55	Fixture Difficulty: 0.27083333333333337
- Cost change: 10
- Score change: 6.47


'New total score: 47.431041666666665'

'New total cost: 993'

'Bench boost not needed. Threshold: 0.409 Average Fixture Difficulty: 0.510'